In [1]:
!pip install pytorch-lightning

     |████████████████████████████████| 686kB 14.3MB/s 
     |████████████████████████████████| 102kB 9.2MB/s 
     |████████████████████████████████| 276kB 43.5MB/s 
     |████████████████████████████████| 829kB 42.3MB/s 
     |████████████████████████████████| 1.3MB 45.8MB/s 
     |████████████████████████████████| 143kB 51.5MB/s 
     |████████████████████████████████| 296kB 46.1MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=dfd024b3ce1c02372a4999e7a9cc88c328248163d24632f9a5b5e665763bd14e
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=d130df80cf192b0142eb8af3046ff7e07efac84e25c8900ae82dff6670c13576
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163

In [2]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import EarlyStopping

def seed():
  pl.utilities.seed.seed_everything(113)

# Data Processing

In [ ]:
!wget https://www.dropbox.com/s/3j76hc0q63it4iu/dataset.zip
!unzip dataset.zip

In [4]:
img_size = 150
train_dir = "train/"
test_dir = "test/"

train_transform = transforms.Compose([
    transforms.CenterCrop(img_size),
    transforms.RandomRotation(5), 
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])


test_transform = transforms.Compose([
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_img = datasets.ImageFolder(train_dir, transform=train_transform)
test_img = datasets.ImageFolder(test_dir, transform=test_transform)

trainloaders = torch.utils.data.DataLoader(train_img, batch_size=64, shuffle=True)
testloaders = torch.utils.data.DataLoader(test_img, batch_size=32)

# Lightning Model

In [5]:
class lightningModel(pl.LightningModule):

    def __init__(self, ks=4, ps=3, fm1=16, fm2=32, n=256, n_labels=5):
        super().__init__()  
        self.conv1 = nn.Conv2d(3, fm1, kernel_size=ks, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=ps, stride=2, padding=0)
        self.conv2 = nn.Conv2d(fm1, fm2, kernel_size=ks, stride=1, padding=0)
        
        # calculate CNN's output size
        res = self.conv_size(self.conv_size(self.conv_size(self.conv_size(img_size, ks), ps, s=2), ks), ps, s=2)**2*fm2
        
        self.fc1 = nn.Linear(res, n)
        self.fc2 = nn.Linear(n, n_labels)
        self.do = nn.Dropout()

        self.loss = nn.CrossEntropyLoss()

    def conv_size(self, inp, k, p=0, s=1):
        return (inp-k+2*p)//s+1

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(x.shape[0], -1)
        x = self.do(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer
    
    def training_step(self, batch, batch_idx):
        image, label = batch
        output = self(image)
        error = self.loss(output, label)
        acc = accuracy(output, label)
        self.log('train_loss', error, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {'loss':error}

    def test_step(self, batch, batch_idx):
        image, label = batch
        output = self(image)
        error = self.loss(output, label)
        acc = accuracy(output, label)
        return {'loss':error, 'accuracy':acc}

    def test_epoch_end(self, outputs):
        acc = torch.Tensor([x['accuracy'] for x in outputs]).mean()
        error = torch.Tensor([x['loss'] for x in outputs]).mean()
        self.log('test_loss', error)
        self.log('test_acc', acc)
        return {'loss':error, 'accuracy':acc}


# Train the Lightning

In [6]:
seed()
model = lightningModel()
trainer = pl.Trainer(gpus=1, max_epochs=20)
trainer.fit(model, trainloaders)

Global seed set to 113
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | conv1 | Conv2d           | 784   
1 | pool  | MaxPool2d        | 0     
2 | conv2 | Conv2d           | 8.2 K 
3 | fc1   | Linear           | 9.5 M 
4 | fc2   | Linear           | 1.3 K 
5 | do    | Dropout          | 0     
6 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
9.5 M     Trainable params
0         Non-trainable params
9.5 M     Total params


1

# Test the Lightning

In [7]:
trainer.test(model, testloaders)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': tensor(0.5677),
 'loss': tensor(1.2040),
 'test_acc': tensor(0.5677),
 'test_loss': tensor(1.2040)}
--------------------------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The testing_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


[{'accuracy': 0.5677083134651184,
  'loss': 1.2040480375289917,
  'test_acc': 0.5677083134651184,
  'test_loss': 1.2040480375289917}]

# Tensorboard for logging

In [8]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/version_4/

<IPython.core.display.Javascript object>